In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.simplefilter('ignore')


In [ ]:
df = pd.read_csv('/ihme/homes/rmbarber/covid-19/elastispliner/2020_06_07.08/model_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['location_id', 'date'])

first_date = df['date'] == df.groupby('location_id')['date'].transform(min)
df['est_daily_death_rate'] = np.diff(df['Predicted death rate (HFR)'], prepend=np.nan)
df['est_daily_deaths'] = df['est_daily_death_rate'] * df['population']
df.loc[first_date, 'est_daily_death_rate'] = df['Predicted death rate (HFR)']
df['daily_hospitalization_rate'] = np.diff(df['Hospitalization rate'], prepend=np.nan)
df.loc[first_date, 'daily_hospitalization_rate'] = df['daily_hospitalization_rate']
df['daily_hospitalizations'] = df['daily_hospitalization_rate'] * df['population']

df['hfr'] = df['daily_hospitalizations'] / df['est_daily_deaths']

df = df.loc[~df['hfr'].isnull()]


In [ ]:
with PdfPages('/ihme/homes/rmbarber/covid-19/hfr_2020_06_07.pdf') as pdf:
    for location_id in df['location_id'].unique():
        plot_df = df.loc[df['location_id'] == location_id].reset_index(drop=True)

        plt.figure(figsize=(11, 8.5))
        plt.plot(plot_df['date'], plot_df['hfr'])
        plt.yticks(fontsize=10)
        plt.xticks(rotation=60, fontsize=10)
        plt.title(plot_df['location_name'][0], fontsize=20)
        plt.ylabel('HFR', fontsize=14)
        plt.xlabel('Date', fontsize=14)
        plt.tight_layout()
        pdf.savefig()

df[['location_id', 'location_name', 'date', 'daily_hospitalizations', 'est_daily_deaths', 'hfr']].to_csv(
    '/ihme/homes/rmbarber/covid-19/hfr_2020_06_07.pdf', index=False
)